# Анализ текстов. Второе задание "Кластеризация писем"

In [1]:
import pandas as pd
import numpy as np

In [2]:
ds = pd.read_csv("Emails.csv")

In [4]:
messages = ds["ExtractedBodyText"]
print len(messages)

7945


Предобработка.
Во-первых, есть много не нужных сообщений.

In [5]:
# todo: mb попробовать отрезать длину?
# texts = [text.decode('utf-8') for text in messages.astype(str).tolist() if (text.lower() != "nan") and (text.lower() != "thx") and (text.lower() != "fyi") ]
texts = [text.decode('utf-8') for text in messages.astype(str).tolist() if (len(text) > 10)]

In [6]:
print texts[:5]
print len(texts)

[u'B6\nThursday, March 3, 2011 9:45 PM\nH: Latest How Syria is aiding Qaddafi and more... Sid\nhrc memo syria aiding libya 030311.docx; hrc memo syria aiding libya 030311.docx\nMarch 3, 2011\nFor: Hillary', u'H <hrod17@clintonemail.com>\nFriday, March 11, 2011 1:36 PM\nHuma Abedin\nFw: H: Latest: How Syria is aiding Qaddafi and more... Sid\nhrc memo syria aiding libya 030311.docx\nPis print.', u"Pis print.\n-\u2022-...-^\nH < hrod17@clintonernailcom>\nWednesday, September 12, 2012 2:11 PM\n\xb0Russorv@state.gov'\nFw: Meet The Right-Wing Extremist Behind Anti-fvluslim Film That Sparked Deadly Riots\nFrom [meat)\nSent: Wednesday, September 12, 2012 01:00 PM\nTo: 11\nSubject: Meet The Right Wing Extremist Behind Anti-Muslim Film That Sparked Deadly Riots\nhtte/maxbiumenthal.com12012/09/meet-the-right-wing-extremist-behind-anti-musiim-tihn-that-sparked-\ndeadly-riots/\nSent from my Verizon Wireless 4G LTE DROID\nU.S. Department of State\nCase No. F-2015-04841\nDoc No. C05739559\nDate: 05/1

Во-вторых, попробуем убрать стоп слова

In [7]:
# Убираем стоп-слова
from nltk.corpus import stopwords
stop_words = stopwords.words("english")
texts_without_stopwords = []
for sample in texts:
    texts_without_stopwords.append(' '.join([word for word in sample.split(' ') if word.lower() not in stop_words]))

In [8]:
print texts_without_stopwords[:5]

[u'B6\nThursday, March 3, 2011 9:45 PM\nH: Latest Syria aiding Qaddafi more... Sid\nhrc memo syria aiding libya 030311.docx; hrc memo syria aiding libya 030311.docx\nMarch 3, 2011\nFor: Hillary', u'H <hrod17@clintonemail.com>\nFriday, March 11, 2011 1:36 PM\nHuma Abedin\nFw: H: Latest: Syria aiding Qaddafi more... Sid\nhrc memo syria aiding libya 030311.docx\nPis print.', u"Pis print.\n-\u2022-...-^\nH < hrod17@clintonernailcom>\nWednesday, September 12, 2012 2:11 PM\n\xb0Russorv@state.gov'\nFw: Meet Right-Wing Extremist Behind Anti-fvluslim Film Sparked Deadly Riots\nFrom [meat)\nSent: Wednesday, September 12, 2012 01:00 PM\nTo: 11\nSubject: Meet Right Wing Extremist Behind Anti-Muslim Film Sparked Deadly Riots\nhtte/maxbiumenthal.com12012/09/meet-the-right-wing-extremist-behind-anti-musiim-tihn-that-sparked-\ndeadly-riots/\nSent Verizon Wireless 4G LTE DROID\nU.S. Department State\nCase No. F-2015-04841\nDoc No. C05739559\nDate: 05/13/2015\nSTATE DEPT. - PRODUCED HOUSE SELECT BENGHAZ

Top bigramms

In [9]:
import nltk
from nltk import word_tokenize
from nltk.util import ngrams
from nltk import FreqDist
tokens=nltk.word_tokenize(''.join(texts_without_stopwords))
bigrams=ngrams(tokens,2)
freq_bigrams = nltk.FreqDist(bigrams)

In [10]:
for bigram, value in freq_bigrams.most_common(30):
    print bigram, value

(u'.', u"''") 1387
(u'.', u'``') 1245
(u',', u"''") 1062
(u'.', u'The') 716
(u'Secretary', u"'s") 676
(u',', u'2010') 596
(u',', u'2009') 573
(u'@', u'state.gov') 534
(u"'s", u'Office') 483
(u'.', u'I') 467
(u'No', u'.') 461
(u'United', u'States') 445
(u'State', u'Department') 440
(u'White', u'House') 411
(u'state.gov', u'>') 399
(u',', u'``') 398
(u'I', u"'m") 374
(u'do', u"n't") 359
(u'Re', u':') 325
(u')', u'.') 296
(u'Department', u'State') 287
(u'it', u"'s") 285
(u')', u',') 282
(u'it', u'.') 276
(u'pm', u'Secretary') 273
(u'Obama', u"'s") 266
(u'.', u'-') 265
(u'.', u'In') 265
(u'U.S.', u'Department') 261
(u',', u'the') 246


Коллокации из 2х слов

In [11]:
from nltk.collocations import BigramAssocMeasures
from nltk.collocations import BigramCollocationFinder
bigram_measures = BigramAssocMeasures()
finder = BigramCollocationFinder.from_words(tokens)
finder.apply_freq_filter(5)

In [12]:
print finder.nbest(bigram_measures.pmi, 20)

[(u'BINYAMIN', u'NETANYAHU'), (u'Chico', u'Menashe'), (u'Deep', u'Throat'), (u'HAMAD', u'BIN'), (u'Im', u'Tirtzu'), (u'Buenos', u'Aires'), (u'Belief', u'Blog'), (u'Elliott', u'Trudeau'), (u'Montreal-Pierre', u'Elliott'), (u'USNS', u'Comfort'), (u'ESMTP', u'id'), (u'Generating', u'server'), (u'Sensitive', u'Unclassified'), (u'Judea', u'Samaria'), (u'Iwo', u'Jima'), (u'Vital', u'Voices'), (u'Cape', u'Verde'), (u'MIDDLE', u'EAST'), (u'Maya', u'Angelou'), (u'Sao', u'Paulo')]


Кластеризация с KMeans

In [23]:
# Признаки
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
vectorizer = TfidfVectorizer(max_df=500, min_df=10, ngram_range=(1, 3))
data_set = [text for text in texts_without_stopwords if len(text) > 10]
matrix = vectorizer.fit_transform(data_set)

In [ ]:
print matrix.shape

(5573, 6091)


In [ ]:
# Кластеризуем
from sklearn.cluster import KMeans
# 5 can interpret
k = 6
model = KMeans(n_clusters=k)
preds = model.fit_predict(matrix.toarray())
print(preds[:30])

In [ ]:
num_of_samples = 10
for j in range(k):
    print "===================================="
    print str(j) + "CLASS:"
    d = model.transform(matrix.toarray())[:, j]
    ind = np.argsort(d)[::][:num_of_samples]
    for i in range(len(ind)):
        print "---------------------------"
        print data_set[ind[i]]